# Practice notebook for confidence intervals using NHANES data

This notebook will give you the opportunity to practice working with confidence intervals using the NHANES data.

You can enter your code into the cells that say "enter your code here", and you can type responses to the questions into the cells that say "Type Markdown and Latex".

Note that most of the code that you will need to write below is very similar to code that appears in the case study notebook.  You will need to edit code from that notebook in small ways to adapt it to the prompts below.

To get started, we will use the same module imports and read the data in the same way as we did in the case study:

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import statsmodels.api as sm

da = pd.read_csv("nhanes_2015_2016.csv")

## Question 1

Restrict the sample to women between 35 and 50, then use the marital status variable [DMDMARTL](https://wwwn.cdc.gov/Nchs/Nhanes/2015-2016/DEMO_I.htm#DMDMARTL) to partition this sample into two groups - women who are currently married, and women who are not currently married.  Within each of these groups, calculate the proportion of women who have completed college.  Calculate 95% confidence intervals for each of these proportions.

In [132]:
dx = da.loc[(da.RIDAGEYR >= 35) & (da.RIDAGEYR <= 50), :] # Restrict to people between 35 and 50 years old
dx = dx.loc[(da.RIAGENDR == 2), :]
#dx = dx.loc[(da.DMDEDUC2 >= 4), :]
dx = dx[["DMDMARTLx", "RIAGENDRx", "DMDEDUC2"]].dropna() 

dx["DMDMARTLx"] = da.DMDMARTL.replace({1: "Married", 2: "Not", 3: "Not", 4: "Not", 5: "Not", 6: "Not", 77: np.nan, 99: np.nan})  # np.nan represents a missing value
#print(dx)

# Summarize the data by calculating the proportion of married and not-married responses and the sample size
p_not_married = dx.groupby("RIAGENDRx")["DMDMARTLx"].agg([lambda z: np.mean(z=="Not"), lambda z: np.mean(z=="Married"), "size"])
p_not_married.columns = ["Not-married", "Married", "N"]
print("\n", p_not_married, "\n")

# Summarize the data by calculating the proportion of women who completed college responses and the sample size
p_college = dx.groupby("DMDMARTLx")["DMDEDUC2"].agg([lambda z: np.mean(z>=4.0), lambda z: np.mean(z<=3.0), "size"])
p_college.columns = ["College", "Not College", "N"]

print("\n", p_college)


print("\n\n")


print("Standard Errors:\n")
p1 = p_college.College.Married # Married & College proportion
n1 = p_college.N.Married # Total number of marrried
se_college_married = np.sqrt(p1 * (1 - p1) / n1)
print("SE of college and married:\n", se_college_married)

p2 = p_college.College.Not # Not married & College proportion
n2 = p_college.N.Not # Total number of not married
se_college_not_married = np.sqrt(p2 * (1 - p2) / n2)
print("\nSE of college and not-married:\n", se_college_not_married)

print("\n\nConfidence intervals:\n")
print("-for married college females:\n")
lcb1 = p1 - 1.96 * np.sqrt(p1 * (1 - p1) / n1)  
ucb1 = p1 + 1.96 * np.sqrt(p1 * (1 - p1) / n1)  
print(lcb1, ucb1)
print("\n-for not-married college females:\n")
lcb2 = p2 - 1.96 * np.sqrt(p2 * (1 - p2) / n2)  
ucb2 = p2 + 1.96 * np.sqrt(p2 * (1 - p2) / n2)
print(lcb2, ucb2)


            Not-married   Married    N
RIAGENDRx                            
Female        0.429479  0.570521  787 


             College  Not College    N
DMDMARTLx                            
Married    0.670379     0.329621  449
Not        0.562130     0.437870  338



Standard Errors:

SE of college and married:
 0.022184241991308255

SE of college and not-married:
 0.026985632877162748


Confidence intervals:

-for married college females:

0.6268975048507106 0.713859733456639

-for not-married college females:

0.5092383370755539 0.6150220179540319


__Q1a.__ Identify which of the two confidence intervals is wider, and explain why this is the case. 

not-married college females has a wider confidence interval because the sample size of not-married females is much smaller than the sample size of married females.

__Q1b.__ Write 1-2 sentences summarizing these findings for an audience that does not know what a confidence interval is (the goal here is to report the substance of what you learned about how marital status and educational attainment are related, not to teach a person what a confidence interval is).

Our sample data shows that of the women who are married, around 67% of them completed college. Whereas around 33% of them did not complete college. Of the women who are not married, 56% of them completed college, whereas around 43% of them did not complete college.
The standard errors and confidence intervals show that the data for married women is more accurate, and this is because the sample size is much bigger. In order to get higher confidence levels for estimates of unmarried women, we must take a larger sample size, preferably closer to the sample size of married women.

## Question 2

Construct 95% confidence intervals for the proportion of smokers who are female, and for the proportion of smokers who are male.  Then construct a 95% confidence interval for the difference between these proportions.

In [156]:
# enter your code here
da["SMQ020x"] = da.SMQ020.replace({1: "Yes", 2: "No", 7: np.nan, 9: np.nan})  # np.nan represents a missing value
da["RIAGENDRx"] = da.RIAGENDR.replace({1: "Male", 2: "Female"})
smokers = da[["RIAGENDRx", "SMQ020x"]].dropna() 
#smokers.columns = ["Gender", "Smoker"]
print(smokers) 

# Summarize the data by calculating the proportion of married and not-married responses and the sample size
smokers = smokers.groupby("RIAGENDRx")["SMQ020x"].agg([lambda z: np.mean(z=="Yes"), "size"])
smokers.columns = ["Smokers", "N"]
print("\n", smokers, "\n")

print("Standard Errors:\n")
p_male_smokers = smokers.Smokers.Male # Male smokers proportion
n_male_smokers = smokers.N.Male # Total number of males
se_male_smokers = np.sqrt(p_male_smokers * (1 - p_male_smokers) / n_male_smokers)
print("SE of male smokers:\n", se_male_smokers)

p_female_smokers = smokers.Smokers.Female # Female smokers proportion
n_female_smokers = smokers.N.Female # Total number of females
se_female_smokers = np.sqrt(p_female_smokers * (1 - p_female_smokers) / n_female_smokers)
print("SE of female smokers:\n", se_female_smokers)

print("\n\nConfidence intervals:\n")
print("-for male smokers:\n")
lcb_male = p_male_smokers - 1.96 * np.sqrt(p_male_smokers * (1 - p_male_smokers) / n_male_smokers)  
ucb_male = p_male_smokers + 1.96 * np.sqrt(p_male_smokers * (1 - p_male_smokers) / n_male_smokers)  
print(lcb_male, ucb_male)
print("\n-for female smokers:\n")
lcb_female = p_female_smokers - 1.96 * np.sqrt(p_female_smokers * (1 - p_female_smokers) / n_female_smokers)  
ucb_female = p_female_smokers + 1.96 * np.sqrt(p_female_smokers * (1 - p_female_smokers) / n_female_smokers)
print(lcb_female, ucb_female)

########

print("\n\nDifferences between genders:\n")
se_diff = np.sqrt(se_female_smokers**2 + se_male_smokers**2)
print("SE difference between genders:", se_diff)

d = smokers.Smokers.Female - smokers.Smokers.Male
lcb_smokers = d - 2*se_diff
ucb_smokers = d + 2*se_diff
print("\nConfidence intervals for difference between genders proportions:\n", lcb_smokers, ucb_smokers)

     RIAGENDRx SMQ020x
0         Male     Yes
1         Male     Yes
2         Male     Yes
3       Female      No
4       Female      No
...        ...     ...
5730    Female     Yes
5731      Male      No
5732    Female     Yes
5733      Male     Yes
5734    Female      No

[5725 rows x 2 columns]

             Smokers     N
RIAGENDRx                
Female     0.304845  2972
Male       0.513258  2753 

Standard Errors:

SE of male smokers:
 0.009526078653689868
SE of female smokers:
 0.008444152146214435


Confidence intervals:

-for male smokers:

0.49458714955108174 0.531929377873546

-for female smokers:

0.288294683866098 0.32139576027925865


Differences between genders:

SE difference between genders: 0.012729881381407434

Confidence intervals for difference between genders proportions:
 -0.2338728044024504 -0.18295327887682067


__Q2a.__ Discuss why it may be relevant to report the proportions of smokers who are female and male, and contrast this to reporting the proportions of males and females who smoke.

__Q2b.__ How does the width of the confidence interval for the difference of the two proportions compare to the widths of the confidence intervals for each proportion separately?

## Question 3

Construct a 95% interval for height ([BMXHT](https://wwwn.cdc.gov/Nchs/Nhanes/2015-2016/BMX_I.htm#BMXHT)) in centimeters.  Then convert height from centimeters to inches by dividing by 2.54, and construct a 95% confidence interval for height in inches.  Finally, convert the endpoints (the lower and upper confidence limits) of the confidence interval from inches to back to centimeters   

In [ ]:
# enter your code here

__Q3a.__ Describe how the confidence interval constructed in centimeters relates to the confidence interval constructed in inches.

## Question 4

Partition the sample based on 10-year age bands, i.e. the resulting groups will consist of people with ages from 18-28, 29-38, etc. Construct 95% confidence intervals for the difference between the mean BMI for females and for males within each age band.

In [ ]:
# enter your code here

__Q4a.__ How do the widths of these confidence intervals differ?  Provide an explanation for any substantial diferences in the confidence interval widths that you see.

## Question 5

Construct a 95% confidence interval for the first and second systolic blood pressure measures, and for the difference between the first and second systolic blood pressure measurements within a subject.

In [ ]:
# enter code here

__Q5a.__ Based on these confidence intervals, would you say that a difference of zero between the population mean values of the first and second systolic blood pressure measures is consistent with the data?


__Q5b.__ Discuss how the width of the confidence interval for the within-subject difference compares to the widths of the confidence intervals for the first and second measures.

## Question 6

Construct a 95% confidence interval for the mean difference between the average age of a smoker, and the average age of a non-smoker.

In [ ]:
# insert your code here

__Q6a.__ Use graphical and numerical techniques to compare the variation in the ages of smokers to the variation in the ages of non-smokers.  

In [1]:
# insert your code here

__Q6b.__ Does it appear that uncertainty about the mean age of smokers, or uncertainty about the mean age of non-smokers contributed more to the uncertainty for the mean difference that we are focusing on here?